# Wikipedia爬蟲練習
## 範例：練習是從Wikipedia中爬取文章。先定義一個搜尋的關鍵字，擷取該關鍵字詞的文章。

In [1]:
import requests
import re
from bs4 import BeautifulSoup

### 先定義一個我們想搜尋的字詞，並將它轉換成UTF-8編碼後的URL

In [2]:
input_keyword = "機器學習"  # 這裡可以自己定義有興趣的關鍵字

utf8_url = repr(input_keyword.encode('UTF-8')).upper()  # 編碼成UTF-8並轉成大寫字元
utf8_url = utf8_url.replace("\\X", "%")                 # 用 '%' 取代 '\X' 
print("%s: %s" % (input_keyword, utf8_url[2:-1:1]))     # 擷取中間的編碼結果

機器學習: %E6%A9%9F%E5%99%A8%E5%AD%B8%E7%BF%92


In [3]:
# 組成Wiki關鍵字搜尋的網址格式
root_keyword_link = './wiki/' + utf8_url[2:-1:1]
print(root_keyword_link)

./wiki/%E6%A9%9F%E5%99%A8%E5%AD%B8%E7%BF%92


### 範例1：送出關鍵字請求後，爬取該關鍵字的文章內容

In [4]:
# 模擬封包的標頭
headers = {
    'authority': 'zh.wikipedia.org',
    'method': 'GET',
    'path': '/wiki/' + root_keyword_link,
    'scheme': 'https',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
    'dnt': '1',
    #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
    'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
}    

url = 'https://zh.wikipedia.org' + root_keyword_link  # 組合關鍵字查詢URL
resp = requests.get(url, headers=headers)
resp.encoding = 'utf-8'

html = BeautifulSoup(resp.text, "lxml")
content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')

#
# 解析回傳資料，並萃取文章內容
#
for paragraph in content:
    print(paragraph.get_text())


機器學習是人工智慧的一個分支。人工智慧的研究歷史有著一條從以「推理」為重點，到以「知識」為重點，再到以「學習」為重點的自然、清晰的脈絡。顯然，機器學習是實現人工智慧的一個途徑，即以機器學習為手段解決人工智慧中的問題。機器學習在近30多年已發展為一門多領域交叉學科，涉及概率論、統計學、逼近論、凸分析（英語：Convex analysis）、計算複雜性理論等多門學科。機器學習理論主要是設計和分析一些讓電腦可以自動「學習」的演算法。機器學習演算法是一類從資料中自動分析獲得規律，並利用規律對未知資料進行預測的演算法。因為學習演算法中涉及了大量的統計學理論，機器學習與推斷統計學聯絡尤為密切，也被稱為統計學習理論。演算法設計方面，機器學習理論關注可以實現的，行之有效的學習演算法。很多推論問題屬於無程式可循難度，所以部分的機器學習研究是開發容易處理的近似演算法。

機器學習已廣泛應用於資料探勘、電腦視覺、自然語言處理、生物特徵辨識、搜尋引擎、醫學診斷、檢測信用卡欺詐、證券市場分析、DNA序列測序、語音和手寫辨識、戰略遊戲和機器人等領域。

機器學習有下面幾種定義：

一種經常參照的英文定義是：A computer program is said to learn from experience E with respect to some class of tasks T and performance measure P, if its performance at tasks in T, as measured by P, improves with experience E.

機器學習可以分成下面幾種類別：

監督學習和非監督學習的差別就是訓練集目標是否人標註。他們都有訓練集 且都有輸入和輸出

具體的機器學習演算法有：





### 範例2：從爬取的文章內容中，擷取出有外部連結的關鍵字。這些關鍵字在文章中是以藍色字體顯示，會連到外部的網頁，並解釋其內容。

In [5]:
for ext_link in content:
    a_tag = ext_link.find_all('a', href=re.compile("^(./wiki/)((?!;)\S)*$"))
    if len(a_tag) > 0:
        for link_string in a_tag:
            a_link = link_string["href"]       # 外部連結的網址
            a_keyword = link_string.get_text()  # 外部連結的中文名稱
            print("外部連結: [%s] %s" % (a_keyword, a_link))

## 作業：接下來定義一個爬蟲函數，這個函數的主要工作為：
### (1) 爬取當前關鍵字的解釋，並存入檔案(因為文章內容太多會佔滿整個頁面，所以存程檔案，方便後續檢視)
### (2) 萃取出當前關鍵字所引用的外部連結，當作新的查詢關鍵字
### (3) 把第(2)擷取到的關鍵字當作新的關鍵字，回到第(1)步，爬取新的關鍵字解釋。

In [8]:
def WikiArticle(key_word_link, key_word, recursive):
    
    if (recursive <= max_recursive_depth):
        print("遞迴層[%d] - %s (%s)" % (recursive, key_word_link, key_word))
        
        # 模擬封包的標頭
        headers = {
            'authority': 'zh.wikipedia.org',
            'method': 'GET',
            'path': './wiki/' + key_word_link,
            'scheme': 'https',
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
            'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
            'dnt': '1',
            #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
            'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
        }    

        url = 'https://zh.wikipedia.org' + key_word_link  # 組合關鍵字查詢URL
        resp = requests.get(url, headers=headers)
        resp.encoding = 'utf-8'

        html = BeautifulSoup(resp.text, "lxml")
        content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')
        
        #
        # Part 1: 請參考範例1，爬取當前關鍵字的文章內容。
        #         因為內容太多，我們把它寫入檔案，並以關鍵字作為檔案名稱，以便稍後查閱內容。
        #         請先建立一個名為"WikiArticle"的資料夾，爬取到的文章內容會放在這個資料夾底下。
        #
        '''
        
        Your code here
        
        '''
        for paragraph in content:  
            file_name = key_word.replace('/', '_')  # 以'_'取代'/', 避免檔名被誤解釋成路徑
            f = open("WikiArticle" + file_name, "a+", encoding='utf-8')
            f.write( str(paragraph.get_text()) + "\n" )
            f.close()
        
        #
        # Part 2: 請參考範例2，萃取出本篇文章中所延伸引用的外部連結，並儲存在external_link_dict
        #
        external_link_dict = dict({})
        '''
        
        Your code here
        
        '''
        for ext_link in content:
            a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)((?!;)\S)*$"))
            if len(a_tag) > 0:
                for link_string in a_tag:
                    a_link = link_string["href"]       # 外部連結的網址
                    a_keyword = link_string.get_text()  # 外部連結的中文名稱
                    #print("外部連結: %s [%s]" % (a_link, a_keyword))
                    external_link_dict[a_link] = a_keyword
                    
        #
        # Part 3: 將Part 2所收集的外部連結，當作新的關鍵字，繼續迭代深入爬蟲
        #
        if (len(external_link_dict) > 0):
            
            recursive = recursive + 1  # 遞迴深度加1
            
            for k, v in external_link_dict.items():
                WikiArticle(k, v, recursive)  # 再次呼叫同樣的函數，執行同樣的流程

### 執行前個步驟定義好的爬蟲主程式

In [9]:
# 定義爬取的遞迴深度。深度不要訂太深，否則會爬很久。
max_recursive_depth = 1

WikiArticle(root_keyword_link, input_keyword, 0)

遞迴層[0] - ./wiki/%E6%A9%9F%E5%99%A8%E5%AD%B8%E7%BF%92 (機器學習)
遞迴層[1] - /wiki/%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD (人工智慧)
遞迴層[1] - /wiki/%E6%8E%A8%E7%90%86 (推理)
遞迴層[1] - /wiki/%E7%9F%A5%E8%AF%86 (知識)
遞迴層[1] - /wiki/%E5%AD%A6%E4%B9%A0 (學習)
遞迴層[1] - /wiki/%E4%BA%A4%E5%8F%89%E5%AD%A6%E7%A7%91 (交叉學科)
遞迴層[1] - /wiki/%E6%A6%82%E7%8E%87%E8%AE%BA (概率論)
遞迴層[1] - /wiki/%E7%BB%9F%E8%AE%A1%E5%AD%A6 (統計學)
遞迴層[1] - /wiki/%E9%80%BC%E8%BF%91%E8%AE%BA (逼近論)
遞迴層[1] - /wiki/%E8%AE%A1%E7%AE%97%E5%A4%8D%E6%9D%82%E6%80%A7%E7%90%86%E8%AE%BA (計算複雜性理論)
遞迴層[1] - /wiki/%E8%AE%A1%E7%AE%97%E6%9C%BA (電腦)
遞迴層[1] - /wiki/%E7%AE%97%E6%B3%95 (演算法)
遞迴層[1] - /wiki/%E6%95%B0%E6%8D%AE (資料)
遞迴層[1] - /wiki/%E8%A7%84%E5%BE%8B (規律)
遞迴層[1] - /wiki/%E6%8E%A8%E6%96%AD%E7%BB%9F%E8%AE%A1%E5%AD%A6 (推斷統計學)
遞迴層[1] - /wiki/%E6%8E%A8%E8%AE%BA (推論)
遞迴層[1] - /wiki/%E6%95%B0%E6%8D%AE%E6%8C%96%E6%8E%98 (資料探勘)
遞迴層[1] - /wiki/%E8%AE%A1%E7%AE%97%E6%9C%BA%E8%A7%86%E8%A7%89 (電腦視覺)
遞迴層[1] - /wiki/%E8%87%AA%E7%84%B6%E8%AF%AD%E8%A8%80%E5%A4%84%E7%90%8

In [13]:
# 定義爬取的遞迴深度。深度不要訂太深，否則會爬很久。
max_recursive_depth = 2

WikiArticle(root_keyword_link, input_keyword, 0)

遞迴層[0] - /wiki/%E6%A9%9F%E5%99%A8%E5%AD%B8%E7%BF%92 (機器學習)
遞迴層[1] - /wiki/%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD (人工智慧)
遞迴層[2] - /wiki/%E9%86%AB%E5%AD%B8 (醫學)
遞迴層[2] - /wiki/%E7%A5%9E%E7%B6%93%E7%A7%91%E5%AD%B8 (神經科學)
遞迴層[2] - /wiki/%E6%9C%BA%E5%99%A8%E4%BA%BA%E5%AD%A6 (機器人學)
遞迴層[2] - /wiki/%E7%B5%B1%E8%A8%88%E5%AD%B8 (統計學)
遞迴層[2] - /wiki/%E4%BA%BA%E9%A1%9E (人類)
遞迴層[2] - /wiki/%E8%81%B7%E6%A5%AD (職業)
遞迴層[2] - /wiki/%E7%BA%A6%E7%BF%B0%C2%B7%E9%BA%A6%E5%8D%A1%E9%94%A1 (約翰·麥卡錫)
遞迴層[2] - /wiki/%E4%BB%BF%E7%94%9F%E5%AD%B8 (仿生學)
遞迴層[2] - /wiki/%E8%AA%8D%E7%9F%A5%E5%BF%83%E7%90%86%E5%AD%B8 (認知心理學)
遞迴層[2] - /wiki/%E6%84%8F%E8%AD%98 (意識)
遞迴層[2] - /wiki/%E8%87%AA%E6%88%91 (自我)
遞迴層[2] - /wiki/%E5%BF%83%E9%9D%88 (心靈)
遞迴層[2] - /wiki/%E6%BD%9B%E6%84%8F%E8%AD%98 (無意識的精神)
遞迴層[2] - /wiki/%E5%8B%95%E7%89%A9 (動物)
遞迴層[2] - /wiki/%E8%A8%88%E7%AE%97%E6%A9%9F (計算機)
遞迴層[2] - /wiki/%E6%A9%9F%E5%99%A8%E4%BA%BA (機器人)
遞迴層[2] - /wiki/%E6%8E%A7%E5%88%B6%E7%B3%BB%E7%B5%B1 (控制系統)
遞迴層[2] - /wiki/%E6%A6%82%E7%8E%87 (概率)


遞迴層[2] - /wiki/%E6%89%B9%E5%88%A4%E7%A4%BE%E5%8D%80%E5%BF%83%E7%90%86%E5%AD%B8 (批判社區心理學)
遞迴層[2] - /wiki/%E6%95%99%E8%82%B2%E5%BF%83%E7%90%86%E5%AD%B8 (教育心理學)
遞迴層[2] - /wiki/%E7%92%B0%E5%A2%83%E5%BF%83%E7%90%86%E5%AD%B8 (環境心理學)
遞迴層[2] - /wiki/%E4%BA%BA%E5%9B%A0%E5%B7%A5%E7%A8%8B%E5%AD%B8 (人因工程學)
遞迴層[2] - /wiki/%E6%B3%95%E5%BA%AD%E5%BF%83%E7%90%86%E5%AD%B8 (法庭心理學)
遞迴層[2] - /wiki/%E5%81%A5%E5%BA%B7%E5%BF%83%E7%90%86%E5%AD%B8 (健康心理學)
遞迴層[2] - /wiki/%E4%BA%BA%E6%9C%AC%E4%B8%BB%E4%B9%89%E5%BF%83%E7%90%86%E5%AD%A6 (人本主義心理學)
遞迴層[2] - /wiki/%E5%B7%A5%E6%A5%AD%E8%88%87%E7%B5%84%E7%B9%94%E5%BF%83%E7%90%86%E5%AD%B8 (工業與組織心理學)
遞迴層[2] - /wiki/%E6%B3%95%E5%BE%8B%E5%BF%83%E7%90%86%E5%AD%A6 (法律心理學)
遞迴層[2] - /wiki/%E7%B6%93%E6%BF%9F%E5%BF%83%E7%90%86%E5%AD%B8 (經濟心理學)
遞迴層[2] - /wiki/%E6%94%BF%E6%B2%BB%E5%BF%83%E7%90%86%E5%AD%B8 (政治心理學)
遞迴層[2] - /wiki/%E5%AE%97%E6%95%99%E5%BF%83%E7%90%86%E5%AD%B8 (宗教心理學)
遞迴層[2] - /wiki/%E9%81%8B%E5%8B%95%E5%BF%83%E7%90%86%E5%AD%B8 (運動心理學)
遞迴層[2] - /wiki/%E4%BA%A4%E9%80%9A

UnicodeEncodeError: 'latin-1' codec can't encode characters in position 40-41: ordinal not in range(256)